In [7]:
import os
import io
import gc
import sys
import yaml
import json
import time
import glob
import wave
import faiss

import random
import librosa

import numpy as np
import pandas as pd
import deepdish as dd
import tensorflow as tf
from pydub import AudioSegment
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from model_RA.fp_RA.nnfp import get_fingerprinter
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer

# **Carregar Faiss Index**

In [2]:
def create_index(db_embeddings, nogpu=True, n_centroids=256, code_sz=64, nbits=8):
    #faiss.IndexIVFPQ(quantizer, d, n_centroids, code_sz, nbits), com d=, nlist=n_centroids=50, m=code_sz=8, bits=nbits=8
    #faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)
    #n_centroids -> clusters

    # GPU Setup
    if not nogpu:
        GPU_RESOURCES = faiss.StandardGpuResources()
        GPU_OPTIONS = faiss.GpuClonerOptions()
        GPU_OPTIONS.useFloat16 = True # use float16 table to avoid https://github.com/facebookresearch/faiss/issues/1178
    else:
        pass
    

    # Fingerprint dimension, d
    d = db_embeddings.shape[1] #len(db_embeddings[0][0]) #db_embeddings.shape[1]  # Dim emb #len(db_embeddings[0][0][0])


    # Build a flat (CPU) index
    quantizer = faiss.IndexFlatL2(d)


    # Using IVF-PQ index
    code_sz = 64 # power of 2
    n_centroids = 256 # Number of Veronoi Cells (?)
    nbits = 8  # nbits must be 8, 12 or 16, The dimension d should be a multiple of M.
    index = faiss.IndexIVFPQ(quantizer, d, n_centroids, code_sz, nbits) #Adicona clustering


    # Se não usar GPU
    if not nogpu:
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)

    max_nitem_train = int(1e7)
    # Train index
    if not index.is_trained:
        #print('Training index using {:>3.2f} % of data...'.format(100. * max_nitem_train / len(db_embeddings)))
        #sel_tr_idx = np.random.permutation(len(db_embeddings))
        #sel_tr_idx = sel_tr_idx[:max_nitem_train]
        #index.train(db_embeddings[sel_tr_idx,:])
        index.train(db_embeddings)
    

    # N probe
    index.nprobe = 40
    return index

# **Carregar Base de Dados**

In [3]:
def def_files(files, train_num):
    embs = []

    for _, file in enumerate(files):
        embs.append(dd.io.load(file))

    embs = np.concatenate(embs, axis=0)
    sel_tr_idx = np.random.permutation(len(embs))
    sel_tr_idx = sel_tr_idx[:train_num]

    return embs, sel_tr_idx

In [4]:
source_dir = '/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/'
h5Files = sorted(glob.glob(os.path.join(source_dir, '**/*.h5') ,recursive = True))


mid_file = len(h5Files)//2
h5Files_1 = h5Files[:mid_file]
h5Files_2 = h5Files[mid_file:]
files = [h5Files_1, h5Files_2]



max_train_num = int(1e7)
idx_1_train = idx_2_train = np.empty((max_train_num//2, 128))


embs_1, idx_1_train = def_files(files[0], max_train_num//2)
print(len(embs_1))
embs_1_sliced = embs_1[idx_1_train,:]
del embs_1, idx_1_train

embs_2, idx_2_train = def_files(files[1], max_train_num//2)
print(len(embs_2))
embs_2_sliced = embs_2[idx_2_train,:]
del embs_2, idx_2_train

embs_sliced = np.concatenate((embs_1_sliced, embs_2_sliced), axis = 0)
del embs_1_sliced, embs_2_sliced

26713592
27040606


In [5]:
%%time
faiss_engine = create_index(embs_sliced, nogpu=True, n_centroids=256, code_sz=64, nbits=8) #dummy_db

CPU times: user 1h 47min 35s, sys: 9.79 s, total: 1h 47min 45s
Wall time: 21min 29s


## **Carrega Informações da Base de Dados**

### **dummy**

In [9]:
%%time
#2h 42min 36s
source_dir = '/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/'
h5Files_dummy = sorted(glob.glob(os.path.join(source_dir, '**/*.h5') ,recursive = True)) #93458 files

#93458 ficheiros/músicas

for idx_file, file in enumerate(h5Files_dummy):
        print(file)
        emb = dd.io.load(file)
        faiss_engine.add(emb)

/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000003.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000020.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000026.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000030.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000046.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000048.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000135.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000136.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000137.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000138.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000139.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000142.h5
/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummyEmb/000/000144.h5
/mnt/dataset/public/Finge

### **db** - músicas sem ruído

In [ ]:
source_dir_db = '/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dbEmb/'
h5Files_db = sorted(glob.glob(os.path.join(source_dir_db, '**/*.h5') ,recursive = True))

#93458 ficheiros/músicas

for idx_file, file in enumerate(h5Files_db):
    print(file)
    emb = dd.io.load(file)
    faiss_engine.add(emb)

In [ ]:
faiss_engine.ntotal

# **Procura no Faiss**

In [ ]:
## Guardar num ficheiro pkl cada segemento com os 100 indíces correspondentes
output_pkl = "/mnt/dataset/public/Fingerprinting/pkl_idxFaiss/"

In [ ]:
%%time
for idx_file, file in enumerate(h5Files_db[:2]):
    print(file)
    emb = dd.io.load(file)
    _, I = faiss_engine.search(q, 100) # D: Distâncias, I: Índices dos resultados

    #Guardar no ficheiro
    base_name = os.path.splitext(os.path.basename(file))[0]
    file_name_pkl = output_pkl + base_name + ".pkl"
    with open(file_name_pkl, 'wb') as file_pkl:
        pickle.dump(I, file_pkl)